In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from PIL import Image


### Split label file to train, val and test

In [ ]:
train_labels = labels.loc[0:5999,:]
train_labels.to_csv('/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/train/labels.csv', sep=',')
val_labels = labels.loc[6000:7999,:]
val_labels.to_csv('/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/val/labels.csv', sep=',')
test_labels = labels.loc[8000:9999,:]
test_labels.to_csv('/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/test/labels.csv', sep=',')



### Customized dataloader for the dataset

In [13]:
class myDataset(Dataset):
    def __init__(self, csv_path, root_dir):
        self.root_dir = root_dir
        
        self.data_info = pd.read_csv(csv_path, header=None)
        #image path
        self.img_array = np.asarray(self.data_info.iloc[1:,1])
        #labels
        self.labels = np.asarray(self.data_info.iloc[1:,2], dtype=np.float32)
        #length of data
        self.data_len = len(self.data_info.index)
        #transformations on the batch
        self.transformations = transforms.Compose([transforms.Resize((256, 256)), transforms.ToTensor()])
    
    def __len__(self):
        return self.data_len
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.img_array[idx] + '.jpeg')
        image = Image.open(img_path,'r')
        img_as_tensor = self.transformations(image)
        
        return {'image':img_as_tensor,'label':self.labels[idx]}
        #return sample
    

In [4]:
csv_path = '/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/train/labels.csv'
root_dir = '/media/eslam/7AE0CA83E0CA455B/Users/Eslam100/Documents/kaggledataset/10k_part/train/'
trainset = myDataset(csv_path, root_dir)

In [6]:
trainloader  = DataLoader(dataset=trainset,
                         batch_size=10,
                         shuffle=False,
                         num_workers=4,
                         pin_memory=True)
for i, batch in enumerate(trainloader):
    print batch['label'].size()
    x = batch['image'].view(batch['image'].size(0), -1)
    print x.size()
    break

torch.Size([10])
torch.Size([10, 196608])


### Plot a batch to make sure it's loaded properly

In [ ]:
trainloader  = DataLoader(dataset=trainset,
                         batch_size=5,
                         shuffle=False,
                         num_workers=4,
                         pin_memory=True)

def plot_batch(sample_batch):
    images_batch = sample_batch['image']
    batch_size = len(images_batch)
    im_size = images_batch.size(2)
    grid = utils.make_grid(images_batch)
    plt.imshow(grid.numpy().transpose((1, 2, 0)))
    plt.title('Batch from dataloader')
    
for i, sample_batch in enumerate(trainloader):
    if i ==3:
        plot_batch(sample_batch)
        plt.show()
        break

### Model architecture

In [11]:
class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 32, 3)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv4 = nn.Conv2d(64, 64, 3)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv5 = nn.Conv2d(64, 128, 3)
        self.conv6 = nn.Conv2d(128, 128, 3)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.conv7 = nn.Conv2d(128, 256, 3)
        self.conv8 = nn.Conv2d(256, 256, 3)
        self.pool4 = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(256*12*12,4096)
        self.fc2 = nn.Linear(4096, 5)
    
    def forward(self, input):
        x = self.pool1(self.conv2(F.relu(self.conv1(input))))
        x = self.pool2(self.conv4(F.relu(self.conv3(x))))
        x = self.pool3(self.conv6(F.relu(self.conv5(x))))
        x = self.pool4(self.conv8(F.relu(self.conv7(x))))
        #flatten the tensor for the FC
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.fc2(x)
        return x
    
    

In [ ]:
use_cuda = torch.cuda.is_available()
learning_rate = 0.0001
l2_penalty = 0.001
batch_size = 5
num_epochs = 60
display_step = 10

trainset = myDataset(csv_path, root_dir)
trainloader  = DataLoader(dataset=trainset,
                         batch_size=1,
                         shuffle=False,
                         num_workers=4,
                         pin_memory=True)
model = ConvNet().cuda()
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate, weight_decay=0.0)
scheduler_model = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=1.0)
criterion = nn.MSELoss(size_average=True).cuda()
losses = []
for epoch in range(num_epochs):
    loss_ = 0.
    predicted = []
    ground_truth = []
    for batch in trainloader:
        data = batch['image']
        labels = batch['label']
        labels.unsqueeze_(0)
        if use_cuda:
            data = data.cuda()
            labels = labels.cuda()
        if labels.data.size()[0] == batch_size:
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, labels)
            loss_ += loss.data[0]
            loss.backward()
            optimizer.step()
    break
    print 'Epoch number:', epoch
    print 'the Loss of the current epoch:', loss_
break